#### Import

In [ ]:
import easyocr
import os
import torch
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os
import random
import glob
from tqdm import tqdm
import numpy as np
import time
from mmocr_eval import eval_ocr_metric
import re
import cv2 as cv
from icecream import ic

In [ ]:
# https://www.kaggle.com/code/fulrose/how-to-apply-new-font-to-matplotlib-easily/notebook
def change_matplotlib_font(font_download_url):
    FONT_PATH = 'MY_FONT'

    # font_download_cmd = f"wget {font_download_url} -O {FONT_PATH}.zip"
    # unzip_cmd = f"unzip -o {FONT_PATH}.zip -d {FONT_PATH}"
    # os.system(font_download_cmd)
    # os.system(unzip_cmd)


    font_files = fm.findSystemFonts(fontpaths=FONT_PATH)
    for font_file in font_files:
        fm.fontManager.addfont(font_file)

    try:
        # print(font_files)
        font_name = fm.FontProperties(fname=font_files[0]).get_name()
    except:
        print('Font not found')
        font_name = 'Kanit'
    matplotlib.rc('font', family=font_name)
    print("font family: ", plt.rcParams['font.family'])

In [ ]:
font_download_url = 'https://fonts.google.com/download?family=Kanit'
change_matplotlib_font(font_download_url)

#### Install packges

In [ ]:
# !pip install matplotlib
# !pip install rapidfuzz

In [ ]:
torch.cuda.is_available()

#### Training

Load model by easyOCR API (https://www.jaided.ai/easyocr/documentation/)

In [ ]:
# start = time.time()
# model = 'rosetta_hippo_300k_v1' #folder name of model
# iteration = 'iter_30000' # model .pt file name

# model_path = os.path.join('trainer', 'saved_models',model)
# reader = easyocr.Reader(lang_list = ['en','th'],
#                     model_storage_directory = model_path,
#                     user_network_directory = os.path.join('my_model','user_network'),
#                     recog_network  = iteration,
#                     config_path = 'crnn', #configuration file name use to get character_list only  (so able to use for every model)
#                     gpu = True)
# # reader = easyocr.Reader(lang_list = ['en','th'],gpu = True )
# end = time.time()
# print(f'loading model time: {end - start}')

visualise testing with some image

In [ ]:
# list_im = [789, 782, 575, 779, 589, 714, 999 ,683]
# filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
# # filepaths = glob.glob(os.path.join('trainer/all_data/training/training','*.jpg'))
# filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))

# font_name = 'Kanit'
# plt.rcParams['xtick.labelsize'] = 20.0
# plt.rcParams['ytick.labelsize'] = 20.0
# all_pred = []
# columns = 4
# rows = 4
# fig = plt.figure(figsize=(15, 15))
# ax = []
# for i in tqdm(range(columns*rows)):
#     # random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
#     # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
#     #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]

#     random_file_path = random.sample(filepaths,1)[0]
#     # random_file_path = filepaths[1]
#     # random_file_path = filepaths[list_im.pop(0)]
#     img = plt.imread(random_file_path)
#     result = reader.recognize(random_file_path)
#     all_pred.append(result[0][1])
#     ax.append(fig.add_subplot(rows, columns, i+1) )
#     boxs = []
#     txt = []
#     for idx,i in enumerate(result):
#         boxs.append(i[0])
#         txt.append(i[1])
#     plt.axis('off')
#     if len(result) > 0:
#         ax[-1].set_title(f'{idx+1} predict: {txt}' , fontsize=20)
#         # for i in boxs:
#         #     pts = np.array(i, np.int32)
#         #     pts = pts.reshape((-1,1,2))
#         #     cv.polylines(img,[pts],True,(0,0,0))

        
#     else:
#         ax[-1].set_title('No result', fontsize=20)
#     plt.imshow(img)
# fig.tight_layout() 
# plt.show()
# print(all_pred)

In [ ]:
# filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
# label_path = 'trainer/all_data/testing/testing/label.txt'
# with open(label_path, encoding='utf8') as file:
#     label = np.loadtxt(file,dtype=str)
# word_correct = 0

In [ ]:
# columns = 5
# rows = 200
# fig, ax = plt.subplots(rows, columns, figsize=(20, 20))
# ax.set_facecolor("white")
# counter = 0
# for r in tqdm(range(rows)):
#     for c in range(columns):
#         file_name = re.split(r'[/\\]',filepaths[counter])[-1]
#         file_path = os.path.join('trainer/all_data/testing/testing',file_name)
#         img = plt.imread(file_path)
#         ax[r, c].axis('off')
#         result = reader.readtext(file_path) 
#         if len(result) > 0:
#             txt = ''
#             for i in result:
#                 txt += (' '+i[1])
#             ax[r, c].set_title(f'{file_name} predict: {txt}')
#         else:
#             ax[r, c].set_title('No result')
#         ax[r, c].imshow(img)
#         counter += 1
# plt.subplots_adjust(top = 20)

testing with all files in dataset

In [ ]:
# filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
# # filepaths = glob.glob(os.path.join('trainer/all_data/training/training','*.jpg'))[:1000]
# filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))
# label_path = 'trainer/all_data/testing/testing/label.txt'
# with open(label_path, encoding='utf8') as file:
#     label = np.loadtxt(file,dtype=str)

# start = time.time()
# # filepaths = filepaths[:3]
# pred = []
# gt = [] # ground truth
# name_img = []
# for i in tqdm(range(len(filepaths))):
#     label_img = label[i][1]
#     name_img.append(label[i][0])
#     result = reader.recognize(filepaths[i],detail=0)
#     try:
#         pred.append(result)
#     except:
#         pred.append('')
#     gt.append(label_img)
# end = time.time()
# print(f'time: {end - start}')

In [ ]:
tmp = name_img.copy()
output, false_list = eval_ocr_metric(pred, gt, tmp)
# output, false_list = eval_ocr_metric(pred, gt, name_img)
for k,v in output.items():
    print(f'{k}: {v}')

visualise testing with flase prediction

In [ ]:

# plt.rcParams['xtick.labelsize'] = 20.0
# plt.rcParams['ytick.labelsize'] = 20.0
# columns = 3
# rows = 5
# fig = plt.figure(figsize=(15, 15))
# ax = []
# all_pred = []
# for i in tqdm(range(columns*rows)):
#     # random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
#     # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
#     #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]

#     random_file_name = random.sample(false_list,1)[0]
#     random_file_name = random_file_name[0]
#     random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)

#     img = plt.imread(random_file_path)
#     result = reader.recognize(random_file_path)
#     ax.append(fig.add_subplot(rows, columns, i+1) )
#     # boxs = []
#     txt = []
#     for idx,i in enumerate(result):
#         # boxs.append(i[0])
#         txt.append(i[1])
#     all_pred.append(txt)
#     plt.axis('off')
#     if len(result) > 0:
#         ax[-1].set_title(f'file: {random_file_name} {idx}: {txt} ', fontsize=20) 
#         # for i in boxs:
#         #         pts = np.array(i, np.int32)
#         #         pts = pts.reshape((-1,1,2))
#         #         cv.polylines(img,[pts],True,(0,0,0))
#     else:
#         ax[-1].set_title(f'file: {random_file_name} No result', fontsize=20)
#     plt.imshow(img)
# fig.tight_layout() 
# plt.show()
# print(all_pred)

testing with LMWN

In [ ]:

# plt.rcParams['xtick.labelsize'] = 20.0
# plt.rcParams['ytick.labelsize'] = 20.0
# columns = 2
# rows = 10
# fig = plt.figure(figsize=(20, 20))
# ax = []
# for i in range(columns*rows):
#     random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/LMWN')),1)[0]
#     # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
#     #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
#     # random_file_name = os.listdir(os.path.join('trainer/all_data/testing/LMWN'))[2]
#     random_file_path = os.path.join('trainer/all_data/testing/LMWN',random_file_name)
#     img = plt.imread(random_file_path)
#     result = reader.recognize(random_file_path) 
#     ax.append(fig.add_subplot(rows, columns, i+1))
#     plt.axis('off')
#     if len(result) > 0:
#         ax[-1].set_title(f'predict: "{result[0][1]}" ', fontsize=10)
#         # pts = np.array(result[0][0], np.int32)
#         # pts = pts.reshape((-1,1,2))
#         # cv.polylines(img,[pts],True,(255,0,0))
#     else:
#         ax[-1].set_title('No result',fontsize=10)
#     plt.imshow(img)
# fig.tight_layout() 
# plt.show()

testing with google ocr testset

In [ ]:
# filepaths = glob.glob(os.path.join('trainer/all_data/testing/google_ocr_test_images','*.jpg'))
# # filepaths = glob.glob(os.path.join('trainer/all_data/training/training','*.jpg'))[:1000]
# filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))
# label_path = 'trainer/all_data/testing/google_ocr_test_images/image.txt'
# label= []
# with open(label_path, encoding='utf8') as file:
#     lines = file.readlines()
#     for line in lines:
#         label.append(eval(line))
# start = time.time()
# # filepaths = filepaths[:3]
# pred = []
# gt = [] # ground truth
# name_img = []
# for i in tqdm(range(len(filepaths))):
#     label_img = label[i]['text']
#     name_img.append(label[i]['filename'])
#     result = reader.recognize(filepaths[i],detail=0)
#     try:
#         pred.append(result)
#     except:
#         pred.append('')
#     gt.append(label_img)
# end = time.time()
# print(f'time: {end - start}')

In [ ]:
# tmp = name_img.copy()
# output, false_list = eval_ocr_metric(pred, gt, tmp)
# # output, false_list = eval_ocr_metric(pred, gt, name_img)
# for k,v in output.items():
#     print(f'{k}: {v}')

In [ ]:

# plt.rcParams['xtick.labelsize'] = 20.0
# plt.rcParams['ytick.labelsize'] = 20.0
# columns = 3
# rows = 5
# fig = plt.figure(figsize=(15, 15))
# ax = []
# all_pred = []
# for i in tqdm(range(columns*rows)):
#     # random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
#     # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
#     #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]

#     random_file_name = random.sample(false_list,1)[0]
#     random_file_name = random_file_name[0]
#     random_file_path = os.path.join('trainer/all_data/testing/google_ocr_test_images',random_file_name)

#     img = plt.imread(random_file_path)
#     result = reader.recognize(random_file_path)
#     ax.append(fig.add_subplot(rows, columns, i+1) )
#     # boxs = []
#     txt = []
#     for idx,i in enumerate(result):
#         # boxs.append(i[0])
#         txt.append(i[1])
#     all_pred.append(txt)
#     plt.axis('off')
#     if len(result) > 0:
#         ax[-1].set_title(f'file: {random_file_name} {idx}: {txt} ', fontsize=15) 
#         # for i in boxs:
#         #         pts = np.array(i, np.int32)
#         #         pts = pts.reshape((-1,1,2))
#         #         cv.polylines(img,[pts],True,(0,0,0))
#     else:
#         ax[-1].set_title(f'file: {random_file_name} No result', fontsize=15)
#     plt.imshow(img)
# fig.tight_layout() 
# plt.show()
# print(all_pred)

In [ ]:
def load_model(model_path, iteration):
    reader = easyocr.Reader(lang_list = ['en','th'],
                    model_storage_directory = model_path,
                    user_network_directory = os.path.join('my_model','user_network'),
                    recog_network  = iteration,
                    config_path = 'crnn', #configuration file name use to get character_list only  (so able to use for every model)
                    gpu = True)
    return reader

In [ ]:
def test_all_testset(reader, test_set):
    filepaths = test_set
    filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))
    label_path = 'trainer/all_data/testing/testing/label.txt'
    with open(label_path, encoding='utf8') as file:
        label = np.loadtxt(file,dtype=str)

    start = time.time()
    # filepaths = filepaths[:3]
    pred = []
    gt = [] # ground truth
    name_img = []
    for i in tqdm(range(len(filepaths))):
        label_img = label[i][1]
        name_img.append(label[i][0])
        result = reader.recognize(filepaths[i],detail=0)
        try:
            pred.append(result)
        except:
            pred.append('')
        gt.append(label_img)
    end = time.time()
    print(f'time: {end - start}')
    return pred, gt, name_img

loop testing

In [ ]:
from IPython.display import clear_output
model = 'rosetta_hippo_300k_v1' #folder name of model
model_path = os.path.join('trainer', 'saved_models',model)
test_set = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
files_list = os.listdir(model_path)

result = {}
for file in files_list:
    if file.startswith('iter'):
        iteration = file.split('.pth')[0]
        reader = load_model(model_path, iteration)
        pred, gt, name_img = test_all_testset(reader, test_set)
        tmp = name_img.copy()
        output, false_list = eval_ocr_metric(pred, gt, tmp)
        # output, false_list = eval_ocr_metric(pred, gt, name_img)
        result[iteration] = output
        clear_output(wait=True)
result

In [ ]:
for k,v in result.items():
    print(f'{k}: {v["word_acc"]}')

In [ ]:
# txt = {'iter_18000': {'char_recall': 0.8387,
#   'char_precision': 0.9379,
#   'word_acc': 0.678,
#   'word_acc_ignore_case': 0.697,
#   'word_acc_ignore_case_symbol': 0.833,
#   '1-N.E.D': 0.8863},
#  'iter_30000': {'char_recall': 0.8719,
#   'char_precision': 0.9815,
#   'word_acc': 0.749,
#   'word_acc_ignore_case': 0.761,
#   'word_acc_ignore_case_symbol': 0.902,
#   '1-N.E.D': 0.9391},
#  'iter_20000': {'char_recall': 0.8552,
#   'char_precision': 0.9726,
#   'word_acc': 0.71,
#   'word_acc_ignore_case': 0.721,
#   'word_acc_ignore_case_symbol': 0.881,
#   '1-N.E.D': 0.9263},
#  'iter_26000': {'char_recall': 0.8516,
#   'char_precision': 0.9774,
#   'word_acc': 0.711,
#   'word_acc_ignore_case': 0.724,
#   'word_acc_ignore_case_symbol': 0.888,
#   '1-N.E.D': 0.9295},
#  'iter_10000': {'char_recall': 0.837,
#   'char_precision': 0.9576,
#   'word_acc': 0.62,
#   'word_acc_ignore_case': 0.638,
#   'word_acc_ignore_case_symbol': 0.851,
#   '1-N.E.D': 0.918},
#  'iter_28000': {'char_recall': 0.8672,
#   'char_precision': 0.973,
#   'word_acc': 0.728,
#   'word_acc_ignore_case': 0.742,
#   'word_acc_ignore_case_symbol': 0.89,
#   '1-N.E.D': 0.9368},
#  'iter_16000': {'char_recall': 0.8461,
#   'char_precision': 0.9736,
#   'word_acc': 0.683,
#   'word_acc_ignore_case': 0.701,
#   'word_acc_ignore_case_symbol': 0.884,
#   '1-N.E.D': 0.9316},
#  'iter_4000': {'char_recall': 0.6886,
#   'char_precision': 0.9279,
#   'word_acc': 0.346,
#   'word_acc_ignore_case': 0.373,
#   'word_acc_ignore_case_symbol': 0.726,
#   '1-N.E.D': 0.8538},
#  'iter_22000': {'char_recall': 0.8487,
#   'char_precision': 0.98,
#   'word_acc': 0.712,
#   'word_acc_ignore_case': 0.725,
#   'word_acc_ignore_case_symbol': 0.894,
#   '1-N.E.D': 0.931},
#  'iter_8000': {'char_recall': 0.7876,
#   'char_precision': 0.949,
#   'word_acc': 0.519,
#   'word_acc_ignore_case': 0.536,
#   'word_acc_ignore_case_symbol': 0.8,
#   '1-N.E.D': 0.8955},
#  'iter_14000': {'char_recall': 0.8393,
#   'char_precision': 0.9623,
#   'word_acc': 0.655,
#   'word_acc_ignore_case': 0.674,
#   'word_acc_ignore_case_symbol': 0.864,
#   '1-N.E.D': 0.9219},
#  'iter_12000': {'char_recall': 0.8231,
#   'char_precision': 0.9652,
#   'word_acc': 0.627,
#   'word_acc_ignore_case': 0.649,
#   'word_acc_ignore_case_symbol': 0.862,
#   '1-N.E.D': 0.9184},
#  'iter_24000': {'char_recall': 0.8666,
#   'char_precision': 0.9801,
#   'word_acc': 0.731,
#   'word_acc_ignore_case': 0.743,
#   'word_acc_ignore_case_symbol': 0.896,
#   '1-N.E.D': 0.9364},
#  'iter_6000': {'char_recall': 0.772,
#   'char_precision': 0.9413,
#   'word_acc': 0.485,
#   'word_acc_ignore_case': 0.512,
#   'word_acc_ignore_case_symbol': 0.785,
#   '1-N.E.D': 0.8779},
#  'iter_2000': {'char_recall': 0.5555,
#   'char_precision': 0.7329,
#   'word_acc': 0.118,
#   'word_acc_ignore_case': 0.122,
#   'word_acc_ignore_case_symbol': 0.542,
#   '1-N.E.D': 0.7264}}

In [ ]:
# import matplotlib.pyplot as plt
# x = []
# y = []
# key_sorted = sorted(txt, key= lambda x: int(x.split('_')[1]))
# for k in key_sorted:
#     x.append(int(k.split('_')[1]))
#     y.append(txt[k]['word_acc'])
# plt.figure(figsize=(8, 6), dpi=80)
# plt.plot(x,y)
# plt.title('Word Accuracy vs Iteration')
# plt.xlabel('Iteration')
# plt.ylabel('Word Accuracy')
# plt.show()